In [3]:
#importing the required modules
import os
import math
import numpy as np
import pandas as pd

In [4]:
home_dir=os.path.join(r'Z:/balint/numt/')

In [212]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
master_array.head()

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,g_sequence,mt_sequence,g_up_flanking,g_down_flanking,"mt_up_flanking (if strand is negative, the coordinates are corrected)","mt_down_flanking (if strand is negative, the coordinates are corrected)",ensembl_gene_id,ensembl_description
0,17245,0.0,0.0,MT,0,0,17245,17245,+,+,17245,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,NaN,NaN,TACAAATTGTATAATATTTGGACTAAACCCCCTTTCCCACCCCAAG...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,ENSOCUG00000029081,NaN
1,3398,0.0,0.0,1,114731432,6976,4680,4681,+,+,194850757,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,CCTTCGCCTCAAAACGAGAAGTAGAGACTATTGAACTAACAACCAC...,AACTGCTAATTCATGCTCCCATGTCTAACCCCATGGCTTTCTCAAC...,ENSOCUG00000006104,cilia and flagella associated protein 300 [Sou...
2,3120,0.0,0.0,GL019308,67160,883,4892,4945,+,+,88228,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...,GATAAACCCCGATAAACTTTACCACTCTTTGCCAACTCAGCCTCTA...,ATGAAGGGGCAAAGAGTAATTTTACAGTCGAGAGGGCTAGCAGTCA...,TACCGCCATCTTCAGCGAACCCTAAAAAGGAGCAAAAGTAAGCTCA...,TAGGGGCTATTAACTTTATTACAACTATTATTAATATGAAAGCCCC...,ENSOCUG00000031196,NaN
3,1888,0.0,0.0,GL019308,62527,12805,3439,3462,+,+,88228,AATTATTCATAGCCTCAATGATGAACAAGACATCCAAAAAACAGGA...,AATTATTCACAGCCTcaaTGATGAACAGGATATCCGAAAGATAGGC...,GTAGGAATTATATCCTTCCTCCTCATCGGCTGATGGCATGGCCAAG...,ACGTATACATGAGTTGTTCTTACATAATTGAATATACCTGCAATGC...,TGCCTAGGCGCAATCACTACCTTATTCACAGCCCTATGTGCCCTCA...,GTACACCCGTACGCACGCACGTACACCCGTACGCACGCACGTACAC...,ENSOCUG00000037635,NaN
4,1824,0.0,0.0,AAGW02081594,14016,6061,3040,3073,+,+,24576,TTCTTTGGACACCCTGAGGTATATATTCTTATCCTTCCAGGATTCG...,TTTTTCGGGCACCCCGAAGTATATATTCTTATTCTTCCAGGATTTG...,GACATGTCAAAGGAACGCAGAAGCCAGCTTGAAAAAGTCCTCAACA...,TGAAATAGCTATCACTATTCAGGTATGCCACTATTCTTCTAGGCAT...,ATATGAAAGCCCCTGCAATATCTCAATATCAAACCCCCTTATTCGT...,ATTCTCCTAGGTATCTATTTTACCCTACTTCAAGCATCGGAGTATT...,NaN,NaN


In [213]:
size=10

In [199]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
chr_mask=master_array['g_id'].apply(lambda g_id: len(g_id) < 3 and g_id!='MT')
master_array=master_array[chr_mask]

In [89]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
scf_mask=master_array['g_id'].apply(lambda g_id: len(g_id) > 3 and g_id!='MT')
master_array=master_array[scf_mask]

In [219]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
gene_mask=master_array['ensembl_gene_id'].apply(lambda g_id: type(g_id)==str)
master_array=master_array[gene_mask]

In [120]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
nongene_mask=master_array['ensembl_gene_id'].apply(lambda g_id: type(g_id)!=str)
master_array=master_array[nongene_mask]

In [141]:
master_array = pd.read_csv(home_dir+'Michael/data/numt_master_array.csv')
size_mask=master_array['g_length']<500
master_array=master_array[size_mask]

In [220]:
master_array

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,g_sequence,mt_sequence,g_up_flanking,g_down_flanking,"mt_up_flanking (if strand is negative, the coordinates are corrected)","mt_down_flanking (if strand is negative, the coordinates are corrected)",ensembl_gene_id,ensembl_description
0,17245,0.000000e+00,0.000000e+00,MT,0,0,17245,17245,+,+,17245,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,NaN,NaN,TACAAATTGTATAATATTTGGACTAAACCCCCTTTCCCACCCCAAG...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,ENSOCUG00000029081,NaN
1,3398,0.000000e+00,0.000000e+00,1,114731432,6976,4680,4681,+,+,194850757,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,CCTTCGCCTCAAAACGAGAAGTAGAGACTATTGAACTAACAACCAC...,AACTGCTAATTCATGCTCCCATGTCTAACCCCATGGCTTTCTCAAC...,ENSOCUG00000006104,cilia and flagella associated protein 300 [Sou...
2,3120,0.000000e+00,0.000000e+00,GL019308,67160,883,4892,4945,+,+,88228,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...,GATAAACCCCGATAAACTTTACCACTCTTTGCCAACTCAGCCTCTA...,ATGAAGGGGCAAAGAGTAATTTTACAGTCGAGAGGGCTAGCAGTCA...,TACCGCCATCTTCAGCGAACCCTAAAAAGGAGCAAAAGTAAGCTCA...,TAGGGGCTATTAACTTTATTACAACTATTATTAATATGAAAGCCCC...,ENSOCUG00000031196,NaN
3,1888,0.000000e+00,0.000000e+00,GL019308,62527,12805,3439,3462,+,+,88228,AATTATTCATAGCCTCAATGATGAACAAGACATCCAAAAAACAGGA...,AATTATTCACAGCCTcaaTGATGAACAGGATATCCGAAAGATAGGC...,GTAGGAATTATATCCTTCCTCCTCATCGGCTGATGGCATGGCCAAG...,ACGTATACATGAGTTGTTCTTACATAATTGAATATACCTGCAATGC...,TGCCTAGGCGCAATCACTACCTTATTCACAGCCCTATGTGCCCTCA...,GTACACCCGTACGCACGCACGTACACCCGTACGCACGCACGTACAC...,ENSOCUG00000037635,NaN
7,1014,0.000000e+00,0.000000e+00,AAGW02081594,17085,9131,1650,1655,+,+,24576,ACTATTCTTCTAGGCATTTATTTTACCCTCCTCCAAGCATCTGAAT...,ACCATTCTCCTAGGTATCTATTTTACCCTACTTCAAGCATCGGAGT...,CCCCAGAACAGGGGGCTGCTGACCTCCACAGGCATTAATCCTCTGA...,CATGTGTTAGCATGTGGCCGGCGCCATGGCTCACTAGGCTAATCCT...,GAGCCTTCTACCACTCAAGCCTAGCTCCTACCCCAGAACTAGGAGG...,GCATGCATAATAGCATTTTTAGTCAAAATACCCTTATATGGGCTTC...,ENSOCUG00000030830,NaN
13,814,0.000000e+00,0.000000e+00,GL018741,2131173,11038,1590,1613,+,+,2352391,CTAAACTCACTCACTGCTTATCCTTCAGTTAGTCATATAGCCTTAG...,CTCAAATCACTCATTGCTTATTCATCAGTGAGCCACATAGCCCTAG...,TCTTAGCACTTTTAGGAATGCTAATCTATCAATCTCAATATCTTCT...,TCTAGAAAACAGGTGTCACAAAATACACGCACTTTGTCCATTACAG...,GATTACCAAAAGCTCACGTTGAAGCCCCCATCGCTGGCTCAATAGT...,CACAAAACGACATCAAAAAAATTGTAGCCTTTTCCACATCAAGTCA...,ENSOCUG00000026879,NaN
20,348,7.600000e-149,1.300000e-152,9,78086277,13328,720,732,+,+,116251907,TTCAACTTCTCTAACCTCTTAGGATTTTTCCCTACAACAATACACC...,TTTAACTTCTCAAACATACTAGGGTTCTTTCCAATGACAATTCATC...,CAAGGTGATCAATGCCTGGGTGTTAGCCCTTAGAGGGTGCAGTGGT...,AGCTACAAATATAACATTTTTATTTATTTATTTATTTTAGAGTTAT...,CTATTAAACGCCTTGCACTAGGAAGCATTTTTGCAGGATTCTTAAT...,CCATCACAAACATTACACTTAATAAAAACACTACGTATGTCATAGT...,ENSOCUG00000000711,KIAA1328 [Source:HGNC Symbol;Acc:HGNC:29248]
21,311,3.100000e-131,5.500000e-135,GL018741,2130623,9583,552,566,+,+,2352391,GAAAAATCAAGCCCCTACGAATGTGGATTTGACCCTATAGGATCAG...,GAAAAATCAAGCCCTTATGAATGCGGATTCGATCCCATAGGATCAG...,AGTCATTTGAGTTAAATTGCACTCAATATTAAGTGTAGTTCTAAAT...,AAACTCACTCACTGCTTATCCTTCAGTTAGTCATATAGCCTTAGGA...,CTATTCCTATATGTATCAATCTATTGATGAGGGTCATACTCTTTTA...,CCAACACTTATGGAATAGACTATGTACAAAACCTAAATCTTCTACA...,ENSOCUG00000026879,NaN
23,267,2.700000e-110,4.800000e-114,12,96965407,17061,419,402,+,+,155355395,TTGGGACTAaacccccctttccccacccccaaaGTCAGGACAGCTC...,TTTGGACTAAACCCCCTTTCCCACCCC----AAGTCAG-ACAGCTC...,CAAATGGATAGATTAAAAAAAAACTAAACTTCAAAAGATGCCAGAA...,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TTTTCCTTTTAACAGGTTTAACTCAATTAAATACAAATTGTATAAT...,ACCACACCCCCAAGGGATACAGCAGTGATAAATATTTAGCAATGAA...,ENSOCUG00000011814,sestrin 1 [Source:HGNC Symbol;Acc:HGNC:21595]
30,222,7.200000e-89,1.300000e-92,9,63094829,13377,397,400,+,+,116

In [221]:
#writing 5 bp upstream and downstream numt sequences into a fasta file
numt_ends=master_array['g_sequence'].apply(lambda sequence: sequence[:size].upper()+sequence[-size:].upper())
numt_ends.index=np.arange(0,len(numt_ends))
with open(home_dir+'numt_g_ends.fasta','w')as outfile:
    numt_ends.apply(lambda seq: outfile.write(seq+'\n'))

In [222]:
#function for flanking preparation
def flanking_prep(row, sequence_type):
    global flankings
    if sequence_type=='mt':
        up=sequence_type+'_up_flanking (if strand is negative, the coordinates are corrected)'
        down=sequence_type+'_down_flanking (if strand is negative, the coordinates are corrected)'
    else:
        up=sequence_type+'_up_flanking'
        down=sequence_type+'_down_flanking'
    if type(row[up])!=str or type(row[down])!=str:
        pass
    else:
        flankings.append(row[up][-size:]+row[down][:size])

In [223]:
flankings=[]
master_array.apply(flanking_prep,args=('g',), axis=1)
g_flankings=pd.Series(flankings)
with open(home_dir+'g_flankings.fasta','w')as outfile:
    g_flankings.apply(lambda seq: outfile.write(seq+'\n'))

In [224]:
flankings=[]
master_array.apply(flanking_prep,args=('mt',), axis=1)
g_flankings=pd.Series(flankings)
with open(home_dir+'mt_flankings.fasta','w')as outfile:
    g_flankings.apply(lambda seq: outfile.write(seq+'\n'))

In [225]:
#writing 5 bp upstream and downstream numt sequences into a fasta file
numt_ends=master_array['mt_sequence'].apply(lambda sequence: sequence[:size].upper()+sequence[-size:].upper())
numt_ends.index=np.arange(0,len(numt_ends))
with open(home_dir+'numt_mt_ends.fasta','w')as outfile:
    numt_ends.apply(lambda seq: outfile.write(seq+'\n'))